In [1]:
# Data parsing and preprocessing into collected df

import pandas as pd
import os

def preprocess(df, patch):
    percentage_columns = ['Win %', 'Role %', 'Pick %', 'Ban %']
    for col in percentage_columns:
        df[col] = df[col].str.rstrip('%').astype(float)

    df['Patch'] = patch
    
    return df

dfs = []
directory = ("./League of Legends Stats S13");
files = os.listdir(directory)

# create combined df from all seasons to have all info together
for file in files:
    if file.endswith(".csv"): 
        file_path = os.path.join(directory, file)
        df = pd.read_csv(file_path, sep=";")

        patch = os.path.splitext(file)[0].split(" ")[-1]
        df = preprocess(df, patch)
        
        dfs.append(df)

# custom sort so the 13.10 patch doesnt come before 13.9 patch, etc.
def custom_sort(patch):
    major, minor = map(int, patch.split('.'))
    return major, minor

combined_df = pd.concat(dfs, ignore_index=True)
combined_df = combined_df.sort_values(by='Patch', key=lambda x: x.map(custom_sort))

# print(combined_df.head())

In [2]:
# Create an overall statistics dataframe and assign color to each champion

import seaborn as sns

tier_map = {'God': 1, 'S': 2, 'A': 3, 'B': 4, 'C': 5, 'D': 6}
role_dfs = []
champion_colors = {}
unique_champions = combined_df['Name'].unique()
distinct_colors = sns.color_palette('hls', n_colors=len(unique_champions))  

# map colors
for champion, color in zip(unique_champions, distinct_colors):
    rgb_color = 'rgb' + str(tuple(int(255 * value) for value in color))
    champion_colors[champion] = rgb_color

# create the average ratings for attributes and create the overall dataframe
for name, role, clss in combined_df[['Name', 'Role', 'Class']].drop_duplicates().values:
    filtered_df = combined_df[(combined_df['Name'] == name) & (combined_df['Role'] == role) & (combined_df['Class'] == clss)]

    numerical_tiers = [tier_map[tier] for tier in filtered_df['Tier']]
    average_numerical_tier = sum(numerical_tiers) / len(numerical_tiers)
    mean_tier = [key for key, value in tier_map.items() if value == round(average_numerical_tier)][0]
    
    mean_score = filtered_df['Score'].mean()
    mean_win_percent = filtered_df['Win %'].mean()
    mean_role_percent = filtered_df['Role %'].mean()
    mean_pick_percent = filtered_df['Pick %'].mean()
    mean_ban_percent = filtered_df['Ban %'].mean()
    mean_kda = filtered_df['KDA'].mean()
    
    role_df = pd.DataFrame({
        'Name': [name],
        'Role': [role],
        'Class': [clss],
        'Mean Tier': [mean_tier],
        'Mean Score': [mean_score],
        'Mean Win %': [mean_win_percent],
        'Mean Role %': [mean_role_percent],
        'Mean Pick %': [mean_pick_percent],
        'Mean Ban %': [mean_ban_percent],
        'Mean KDA': [mean_kda],
        'Color': [champion_colors[name]]
    })
    
    role_dfs.append(role_df)

overall_df = pd.concat(role_dfs, ignore_index=True)

# print(overall_df.sort_values('Name'))

In [3]:
# Sorted stream graph
# y axis - primary attribute
# thickness of the stream - secondary attribute
# can pick to show champions depeding on role or class (initially by role, top) - tertiary attribue

import plotly.graph_objects as go
import seaborn as sns
import math

def main_stream_graph(primary_attr, secondary_attr, tertiary_attr, tertiary_attr_value, champions_to_show=10):
    # filter dfs based off of the tertiary attribute value and merge
    if(tertiary_attr == 'All'):
        tert_df = combined_df.merge(overall_df, on='Name')
    else:
        tert_df = combined_df[combined_df[tertiary_attr] == tertiary_attr_value].merge(overall_df[overall_df[tertiary_attr] == tertiary_attr_value], on='Name')
    sorted_unique_champions = tert_df.groupby('Name', as_index=False).first().sort_values(by=primary_attr, ascending=False)['Name'].tolist()

    # thickness setting for secondary attribute
    min_rate_rounded = 5 * round(tert_df[secondary_attr].min()/5) - 5
    max_rate_rounded = 5 * round(tert_df[secondary_attr].max()/5)

    # find the most rated champion for each patch by primary attribute and sort
    most_pr_rated_champions = tert_df.loc[tert_df.groupby('Patch',sort=False)[primary_attr].idxmax(), ['Patch', 'Name']]
    most_pr_rated_champions_dict = dict(zip(most_pr_rated_champions['Patch'], most_pr_rated_champions['Name']))

    patch_order = combined_df['Patch'].unique().tolist()
    most_pr_rated_champions_dict = {k: most_pr_rated_champions_dict[k] for k in patch_order if k in most_pr_rated_champions_dict}

    traces = []
    for champion_name in tert_df['Name'].unique().tolist():
        champion_data = tert_df[tert_df['Name'] == champion_name]

        # only show n overally most rated champions based on the primary attribute
        if champion_name in sorted_unique_champions[:champions_to_show] or champion_name in most_pr_rated_champions_dict.values():
            visible = True
        else:
            visible = 'legendonly'

        # normalize secondary attribute to range 0.0 - 1.0
        normalized_rate_value = (champion_data[secondary_attr].iloc[0] - min_rate_rounded) / (max_rate_rounded - min_rate_rounded)
                
        trace = go.Scatter(
            x=champion_data['Patch'],
            y=champion_data[primary_attr],
            mode='lines+markers',
            name=champion_name,
            visible=visible,
            line=dict(
                color=champion_data['Color'].iloc[0],
                width=normalized_rate_value * 10
            ),
            marker=dict(
                size=normalized_rate_value * 40,
                line=dict(width=0, color='White')
            ),
            opacity=0.5
        )
        traces.append(trace)

    annotations = []
    for patch, champion in most_pr_rated_champions_dict.items():
        annotation = go.layout.Annotation(
            x=list(most_pr_rated_champions_dict).index(patch),
            y=int(math.ceil(tert_df[primary_attr].max()))+1,
            xref='x',
            yref='y',
            text=champion,
            ax=0,
            ay=0,
            font=dict(size=10)
        )
        annotations.append(annotation)
    
    layout = go.Layout(
        title='Champions ' + primary_attr + ' Over Patches - ' + tertiary_attr + ': ' + (tertiary_attr_value if tertiary_attr != 'All' else ''),
        legend_title_text='Champions',
        xaxis=dict(title='Patch'),
        yaxis=dict(title=primary_attr),
        template='plotly_dark',
        width=1800,
        height=800,
        annotations=annotations
    )

    fig = go.Figure(data=traces, layout=layout)

    # remove duplicate legend entries
    champion_names = set()
    fig.for_each_trace(
        lambda trace:
            trace.update(showlegend=False)
            if (trace.name in champion_names) else champion_names.add(trace.name))

    #fig.show(renderer='iframe')
    return fig    

#main_stream_graph('Pick %', 'Win %', 'Role', 'SUPPORT')

In [4]:
# Pie diagrams for specific champion

import plotly.subplots as ps

prate_labels = ["Picked", "Not picked"]
brate_labels = ["Banned", "Not banned"]
wrate_labels = ["Win", "Loss"]
role_labels = ["ADC", "JUNGLE", "MID", "SUPPORT", "TOP"]
patch_titles = combined_df['Patch'].unique().tolist()

role_colors = ['royalblue', 'tomato', 'lightseagreen', 'mediumpurple', '#8D6E63']

def calculate_values(attribute, champion_name, patch=""):
    if(patch != ""):
        df = combined_df
    else:
        df = overall_df
        
    champion_df = df[df['Name'] == champion_name]
    values = []
    # values for roles
    if(attribute == 'Role'):
        if(patch != ""):
            pick = 'Pick %'
            role_picks_df = champion_df[champion_df['Patch'] == patch][['Role', pick]]
        else:
            pick = 'Mean Pick %'
            role_picks_df = champion_df[['Role', pick]]
        role_picks_df = role_picks_df.groupby('Role')[pick].mean().reset_index()
        
        for role in role_labels:
            if(role not in role_picks_df['Role'].tolist()):
                role_picks_df.loc[len(role_picks_df.index)] = [role, 0]
                
        role_picks_df = role_picks_df.sort_values('Role')
        values = role_picks_df[pick].tolist()
    # values for numerical attributes
    else:
        if(patch != ""):
            mean_rate = champion_df[champion_df['Patch'] == patch][attribute].mean()
        else:
            mean_rate = champion_df[attribute].mean()
        values = [mean_rate, 100-mean_rate]
    return values

# get list of tiers to be able to show custom labels on the last subplot for role
def calculate_tiers(champion_name, patch=""):
    champion_df = combined_df[(combined_df['Name'] == champion_name) & (combined_df['Patch'] == patch)]
    column = 'Tier'
    if(patch == ""):
        champion_df = overall_df[(overall_df['Name'] == champion_name)]
        column = 'Mean Tier'

    tiers_dict = dict.fromkeys(role_labels)
    for role, tier in tiers_dict.items():
        if role in champion_df['Role'].tolist():
            tiers_dict[role] = champion_df[champion_df['Role'] == role][column].iloc[0]
        else:
            tiers_dict[role] = None

    return list(tiers_dict.values())

def pie_charts(attribute, overall_attribute, labels, champion_name):
    titles = patch_titles + ['Season 13']
    fig = ps.make_subplots(rows=1, cols=13, subplot_titles=titles, specs=[[{'type':'domain'}]*13], column_widths=[0.2]*12 + [0.3])

    # make first 12 pie charts, one for each patch
    for patch in patch_titles:
        text = []
        if(attribute == 'Role'):
            tiers = calculate_tiers(champion_name, patch)
            text=tiers
        values = calculate_values(attribute, champion_name, patch)
        col = patch_titles.index(patch) + 1
        fig.add_trace(go.Pie(labels=labels, values=values, name=patch, texttemplate=text, textfont= {'size': [15]*5, 'color':'white'}, direction='clockwise', sort=False), row=1, col=col)

    # make the last pie chart for overall season 13 stats
    if(attribute == 'Role'):
        tiers = calculate_tiers(champion_name)
        text = tiers
    else:
        text = []
    values = calculate_values(overall_attribute, champion_name)
    fig.add_trace(go.Pie(labels=labels, values=values, name='overall s13', texttemplate=text, textfont={'size': [15]*5, 'color':'white'}, direction='clockwise', sort=False), row=1, col=13)

    fig.update_traces(hole=.4, hoverinfo="label+percent+name", textinfo = 'none', marker=dict(colors=role_colors))
    fig.update_layout(template='plotly_dark',height=300,width=1950,title_text=attribute + ' over the season:', margin=dict(l=20, r=20, t=10, b=0),title_y=0.98,title_x=0.028)
    fig.update_annotations(yshift=-50)
    # fig.show(renderer='iframe')
    return fig

# pie_charts('Pick %', 'Mean Pick %', prate_labels, "Lux")
# pie_charts('Win %', 'Mean Win %', wrate_labels, "Lux")
# pie_charts('Ban %', 'Mean Ban %', brate_labels, "Lux")
# pie_charts('Role', 'Role', role_labels, "Lux")

In [ ]:
# Main Dash app

import dash
from dash import dcc, html, Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.graph_objs as go
import copy
import pandas as pd
from dash import callback_context

# set initial graph
initial_primary = 'Pick %'
initial_secondary = 'Win %'
initial_tertiary = 'Role'
initial_tertiary_value = 'SUPPORT'
initial_fig = main_stream_graph(initial_primary, initial_secondary, initial_tertiary, initial_tertiary_value)

app = dash.Dash(__name__)

# define the app layout
app.layout = html.Div(
    style={'backgroundColor': '#111111',
        'height': '100%',
        'color': 'white',
        'margin': 0,
        'padding': '15px' 
    },
    children=[
    html.H1("League of Legends Season 13 Statistics"),
    html.H2("Analysis over the patches"),
    # main graph menu div, dropdowns and button
    html.Div([ 
        html.Div([
            dcc.Markdown('Primary rating (Y axis)'),
            dcc.Dropdown(
                id='primary-dropdown',
                options=[
                    {'label': 'Pick Rate', 'value': 'Pick %'},
                    {'label': 'Win Rate', 'value': 'Win %'},
                    {'label': 'Ban Rate', 'value': 'Ban %'},
                ],
                value=initial_primary,
                searchable=False,
                style={'color': 'black'}
            )
        ], style={'width': '15%', 'display': 'inline-block', 'margin-right': '2%'}),
        html.Div([
            dcc.Markdown('Secondary rating (Line/marker size)'),
            dcc.Dropdown(
                id='secondary-dropdown',
                options=[
                    {'label': 'Pick Rate', 'value': 'Pick %'},
                    {'label': 'Win Rate', 'value': 'Win %'},
                    {'label': 'Ban Rate', 'value': 'Ban %'},
                ],
                value=initial_secondary, 
                searchable=False,
                style={'color': 'black'}
            )
        ], style={'width': '15%', 'display': 'inline-block', 'margin-right': '2%'}),
        html.Div([
            dcc.Markdown('Occupation type'),
            dcc.Dropdown(
                id='tertiary-dropdown',
                options=[
                    {'label': 'Class', 'value': 'Class'},
                    {'label': 'Role', 'value': 'Role'},
                    {'label': 'All', 'value': 'All'}
                ],
                value=initial_tertiary, 
                searchable=False,
            style={'color': 'black'})
        ], style={'width': '15%', 'display': 'inline-block', 'margin-right': '2%'}),
        html.Div([
            dcc.Markdown('Occupation'),
            dcc.Dropdown(
                id='tertiary-value-dropdown',
                value=initial_tertiary_value, 
                searchable=False,
                style={'color': 'black'})
        ], style={'width': '15%', 'display': 'inline-block', 'margin-right': '2%'}),
        html.Button('Update', id='update-button', n_clicks=0, style={'width': '150px', 'height': '50px', 'margin-top': '50px'}),
    ], style={'display': 'flex', 'align-items': 'center', 'margin-bottom': '1%', 'width': '1800px'}),
    # main graph div
    html.Div([
        dcc.Graph(id='main-graph', figure=initial_fig), 
        html.Div(id='champion-info-tooltip')
    ], style={'display': 'flex'}),
    # slider div
    html.Div([
        dcc.Slider(
            id='champions-slider',
            min=1,
            max=30,
            step=1,
            value=10,
            marks={i: str(i) for i in range(1, 31)},
            tooltip={'placement': 'bottom'}
        )
    ], style={'width': '1800px', 'padding-top': '20px', 'margin-bottom':'50px'}),
    # champion analysis menu div
    html.Div([
            html.H2("Champion Analysis"),
            dcc.Input(
                id='search-champion',
                type='text',
                placeholder='Champion name...',
                style={'width': '200px', 'margin-right': '10px'}
            ),
            html.Button('Show Champion Stats', id='show-info-button', n_clicks=0, style={'height': '30px'}),
    ], style={'margin-top': '20px'}),
    # champion analysis graphs div
    html.Div(id='champion-info-container')
])

# update dropdown for tertiary attribute values
@app.callback(
    Output('tertiary-value-dropdown', 'options'),
    [Input('tertiary-dropdown', 'value')]
)
def update_tertiary_value_dropdown(selected_tertiary):
    if selected_tertiary == 'All':
        return []
    if selected_tertiary is None:
        raise PreventUpdate
    unique_values = combined_df[selected_tertiary].unique()
    options = [{'label': value, 'value': value} for value in unique_values]
    return options

# triggers when clicking on the update button
@app.callback(
    Output('main-graph', 'figure', allow_duplicate=True),
    [Input('update-button', 'n_clicks')],
    [State('primary-dropdown', 'value')],
    [State('secondary-dropdown', 'value')],
    [State('tertiary-dropdown', 'value')],
    [State('tertiary-value-dropdown', 'value')],
    [State('champions-slider', 'value')],
    prevent_initial_call=True
)
def update_main_graph(n_clicks, primary_attr, secondary_attr, tertiary_attr, tertiary_attr_value, champions_to_show):
    if n_clicks == 0:
        raise PreventUpdate

    fig = main_stream_graph(primary_attr, secondary_attr, tertiary_attr, tertiary_attr_value, champions_to_show)

    return fig

# callback for the champion analysis visualization
@app.callback(
    Output('champion-info-container', 'children', allow_duplicate=True),
    [Input('show-info-button', 'n_clicks')],
    [State('search-champion', 'value')],
    prevent_initial_call=True
)
def display_champion_info(n_clicks, champion_name):
    if n_clicks == 0:
        raise PreventUpdate

    pie_chart_1 = pie_charts('Win %', 'Mean Win %', wrate_labels, champion_name)
    pie_chart_2 = pie_charts('Pick %', 'Mean Pick %', prate_labels, champion_name)
    pie_chart_3 = pie_charts('Ban %', 'Mean Ban %', brate_labels, champion_name)
    pie_chart_4 = pie_charts('Role', 'Role', role_labels, champion_name)
    
    return html.Div([
        html.Div([
            html.H3(champion_name + " (" + combined_df[combined_df['Name'] == champion_name]['Class'].iloc[0] + ")"),
        ], style={'margin-top':'0.5'}), 
        dcc.Graph(id='pie-chart-1', figure=pie_chart_1),
        dcc.Graph(id='pie-chart-2', figure=pie_chart_2),
        dcc.Graph(id='pie-chart-3', figure=pie_chart_3),
        dcc.Graph(id='pie-chart-4', figure=pie_chart_4)
    ])

# triggers when clicking on trace marker
@app.callback(
    Output('main-graph', 'figure', allow_duplicate=True),
    Output('champion-info-container', 'children', allow_duplicate=True),
    Output('champion-info-tooltip', 'children'),
    [Input('main-graph', 'clickData')],
    [State('main-graph', 'figure')],
    [State('primary-dropdown', 'value')],
    [State('secondary-dropdown', 'value')],
    [State('tertiary-dropdown', 'value')],
    [State('tertiary-value-dropdown', 'value')],
    prevent_initial_call=True
)
def update_trace(click_data, figure, primary_attr, secondary_attr, tertiary_attr, tertiary_attr_value):
    if not click_data:
        raise PreventUpdate

    if click_data:
        clicked_line_index = click_data['points'][0]['curveNumber']
        clicked_point_index = click_data['points'][0]['pointIndex']

        # set traces opacities and outline the selected marker
        for i, trace in enumerate(figure['data']):
            trace['opacity'] = 0.5
            if i == clicked_line_index:
                trace['marker']['line']['width'] = [2 if j == clicked_point_index else 0 for j in range(len(trace['x']))]
            else:
                trace['marker']['line']['width'] = 0
        figure['data'][clicked_line_index]['opacity'] = 1.0

        if tertiary_attr == 'All':
            tert_df = combined_df.merge(overall_df, on='Name')
        else:
            tert_df = combined_df[combined_df[tertiary_attr] == tertiary_attr_value].merge(overall_df[overall_df[tertiary_attr] == tertiary_attr_value], on='Name')
        
        # call display_champion_info callback with the name of the clicked trace and show tooltip on the side of the main graph
        ctx = callback_context
        triggered_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if triggered_id == 'main-graph':
            champion_name = figure['data'][clicked_line_index]['name']
            champion_info = display_champion_info(1, champion_name)
            patch = click_data['points'][0]['x']
            tooltip = html.Div([
                dcc.Markdown('Extended tooltip:'),
                html.Div([
                    dcc.Markdown('Champion: ' + figure['data'][clicked_line_index]['name']),
                    dcc.Markdown('Patch: ' + patch),
                    dcc.Markdown(primary_attr + ": " + str(click_data['points'][0]['y'])),
                    dcc.Markdown(secondary_attr + ": " + str(tert_df[(tert_df['Name'] == champion_name) & (tert_df['Patch'] == patch)][secondary_attr].iloc[0])) #rename these two to fit the attributes
                ], style={'marginLeft': '5px', 'marginRight': '5px'})
            ], style={'margin-left': '20px',  'margin-top': '110px', 'border': '1px solid white', 'padding': '5px', 'position': 'relative'})
            return figure, champion_info, tooltip

    return figure, dash.no_update, dash.no_update
    
if __name__ == '__main__':
    app.run_server(jupyter_mode="external", debug=True)